# Способы извлечь сущности

1. Взять родовое понятие, обучить word2vec на текстах отзывах (мне кажется важно обучить модель именно на текстах ревью из которых будем извлекать сущности), получить близкие к родовому слову синонимы и доставать из текстов эти слова. Минусы: не удастся вытащить прям все, потому что в отзывах могут встречаться не только родовые понятия, но и названия фильмов.
2. Взять из метадаты названия и добавить их в словарь к родовым понятиям. Плюсы: высокая полнота. Минусы: очень много ручной работы, потому что в названиях есть подзаголовки, части, франшиза. И любые эти части могут использоваться в отзыве, скорее всего придется держать словарь, где каждому названию будут соответствовать разные варианты его написания (сокращения, подзаголовок, полное название и т.д.)
3. Взять родовые понятия в word2vec, взять синонимы к ним, заменить их на какой-нибудь тэг типа NE. Обучить модель (хотя бы перцептрон) предсказывать по контексту будет ли там NE или нет. Плюсы: так можно предсказывать также названия (потому что они встречаются в тех же контекстах, что и родовые понятия). Минусы: нужно будет что-то придумать с длинной NE, так как названия могут состоять из нескольких слов.

Я выбрал первый способ и 5-core [датасет](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz) Movies and TV


In [1]:
import json
import nltk
import gzip
import re
import gensim
import pandas as pd
import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from nltk.collocations import *
from string import punctuation
from collections import Counter

lemmatizer = WordNetLemmatizer()

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
reviews = getDF('Movies_and_TV_5.json.gz')

In [46]:
def preprocess(text, stopwords, delete_stopwords=False):
    text = text.lower().strip()
    for char in punctuation:
        text = text.replace(char, " ")
    tokens = text.split()
    new_text = []
    for i in tokens:
        if delete_stopwords:
            if i not in stopwords:
                new_text.append(i)
        else:
            new_text.append(i)
    return new_text
stop = stopwords.words('english')

In [47]:
reviews = shuffle(reviews)

In [48]:
corpus = [preprocess(i, stop) for i in reviews[:300000].reviewText.tolist()]

In [50]:
model = gensim.models.Word2Vec(corpus, window=2)

Возьмем в качестве родовых понятия movie, series, video и film и достанем синонимы к ним

In [51]:
entities = []
for i in model.wv.most_similar('movie', topn=15):
    entities.append(i[0])

In [52]:
for i in model.wv.most_similar('series', topn=15):
    entities.append(i[0])

In [53]:
for i in model.wv.most_similar('video', topn=15):
    entities.append(i[0])

In [54]:
for i in model.wv.most_similar('film', topn=15):
    entities.append(i[0])

In [55]:
entities = [lemmatizer.lemmatize(i) for i in entities]

In [56]:
entities = set(entities)
entities = entities.difference(['lp', 'bootleg', 'workout', 'installment', 'match', 'it', 'thing', 'story', 
                               'tape', 'cd', 'bd', 'dvd', 'vhs', 'tape', 'recording',
                               'laserdisc', 'concert', 'album', 'lp', 'product', 'storyline', 'boxset', 'demo',
                               'genre', 'bluray', 'playback', 'hd', 'youtube', 'concert'])
entities = list(entities)

In [57]:
def get_mention(text, entities):
    mentions = []
    for word in text.split():
        if lemmatizer.lemmatize(word) in entities:
            mentions.append(word)
    return mentions

In [58]:
new_corpus = []

for i in corpus:
    new_corpus.append(' '.join(i))

In [59]:
for i in range(len(new_corpus)):
    new_corpus[i] = ' '.join(preprocess(new_corpus[i], stop, True))

In [60]:
bigrams = ngrams(' '.join(new_corpus).split(), 2)

# N-граммы

In [61]:
bi = Counter(bigrams)

In [63]:
bi_entities = []
for i in bi.most_common():
    if lemmatizer.lemmatize(i[0][0]) in entities or lemmatizer.lemmatize(i[0][1]) in entities:
        bi_entities.append(' '.join([i[0][0], i[0][1]]))

In [64]:
trigrams = ngrams(' '.join(new_corpus).split(), 3)
tri = Counter(trigrams)
tri_entities = []
for i in tri.most_common():
    if lemmatizer.lemmatize(i[0][0]) in entities or lemmatizer.lemmatize(i[0][1]) in entities or lemmatizer.lemmatize(i[0][2]) in entities:
        tri_entities.append(' '.join([i[0][0], i[0][1], i[0][2]]))

# Коллокации

In [65]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(' '.join(new_corpus).split())
finder.apply_freq_filter(20)

In [66]:
pmi = []
for item in entities:
    rate = [i for i in finder.nbest(bigram_measures.pmi, 1000000) if item in i]
    pmi.append([' '.join(i) for i in rate])

In [67]:
pmi[0][:30]

['mbpsaverage video',
 'video codec',
 'muffled video',
 'instant video',
 'instructional video',
 'alpha video',
 'video games',
 'video store',
 'direct video',
 'video stores',
 'video game',
 'video library',
 'kino video',
 'video tape',
 'video audio',
 'streaming video',
 'video taped',
 'home video',
 'video diary',
 'video demand',
 'video transfer',
 'audio video',
 'video quality',
 'video clips',
 'video rental',
 'straight video',
 'mtv video',
 'video cameras',
 'video tapes',
 'music video']

In [68]:
jaccard = []
for item in entities:
    rate = [i for i in finder.nbest(bigram_measures.jaccard, 1000000) if item in i]
    jaccard.append([' '.join(i) for i in rate])

In [69]:
jaccard[0][:30]

['video game',
 'video quality',
 'home video',
 'video store',
 'video audio',
 'video games',
 'music video',
 'audio video',
 'direct video',
 'instant video',
 'video transfer',
 'video tape',
 'video library',
 'straight video',
 'local video',
 'quality video',
 'video collection',
 'video release',
 'video camera',
 'alpha video',
 'video sound',
 'video rental',
 'sound video',
 'video stores',
 'video releases',
 'concert video',
 'video fair',
 'video clips',
 'video footage',
 'released video']

In [70]:
dice = []
for item in entities:
    rate = [i for i in finder.nbest(bigram_measures.dice, 1000000) if item in i][:150]
    dice.append([' '.join(i) for i in rate])

In [71]:
dice[0][:30]

['video game',
 'video quality',
 'home video',
 'video store',
 'video audio',
 'video games',
 'music video',
 'audio video',
 'direct video',
 'instant video',
 'video transfer',
 'video tape',
 'video library',
 'straight video',
 'local video',
 'quality video',
 'video collection',
 'video release',
 'video camera',
 'alpha video',
 'video sound',
 'video rental',
 'sound video',
 'video stores',
 'video releases',
 'concert video',
 'video fair',
 'video clips',
 'video footage',
 'released video']

In [72]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(' '.join(new_corpus).split())
finder.apply_freq_filter(20)

In [73]:
pmi_tri = []
for item in entities:
    rate = [i for i in finder.nbest(trigram_measures.pmi, 1000000000) if item in i][:150]
    pmi_tri.append([' '.join(i) for i in rate])
pmi_tri[0][:30]

['video resolution codec',
 'fair muffled video',
 'mbpsaverage video bit',
 'mpi home video',
 'amazon instant video',
 'prime instant video',
 'local video store',
 'video rental store',
 'warner home video',
 'fair video fair',
 'audio fair video',
 'direct video sequel',
 'video audio quality',
 'audio video quality',
 'direct video release',
 'playing video games',
 'video bit rate',
 'based video game',
 'based video games',
 'went straight video',
 'play video games',
 'blu ray video',
 'home video release',
 'video sound quality',
 'quality video audio',
 'highly recommend video',
 'sound video quality',
 'released home video',
 'video quality blu',
 'quality audio video']

In [74]:
jaccard_tri = []
for item in entities:
    rate = [i for i in finder.nbest(trigram_measures.jaccard, 1000000000) if item in i][:150]
    jaccard_tri.append([' '.join(i) for i in rate])
jaccard_tri[0][:30]

['amazon instant video',
 'warner home video',
 'local video store',
 'video audio quality',
 'audio video quality',
 'audio fair video',
 'blu ray video',
 'fair video fair',
 'based video game',
 'prime instant video',
 'video rental store',
 'video bit rate',
 'fair muffled video',
 'home video release',
 'video sound quality',
 'video resolution codec',
 'highly recommend video',
 'direct video sequel',
 'mbpsaverage video bit',
 'sound video quality',
 'mpi home video',
 'video quality excellent',
 'video blu ray',
 'video quality blu',
 'went straight video',
 'quality video audio',
 'video game series',
 'direct video release',
 'playing video games',
 'released home video']

In [75]:
likelihood_ratio = []
for item in entities:
    rate = [i for i in finder.nbest(trigram_measures.likelihood_ratio, 1000000000) if item in i][:150]
    likelihood_ratio.append([' '.join(i) for i in rate])
likelihood_ratio[0][:30]

['blu ray video',
 'video blu ray',
 'highly recommend video',
 'would recommend video',
 'video sound quality',
 'audio video quality',
 'video quality dvd',
 'based video game',
 'video game series',
 'like video game',
 'video game movie',
 'warner home video',
 'video audio quality',
 'recommend video anyone',
 'video quality excellent',
 'sound video quality',
 'video quality blu',
 'video quality good',
 'local video store',
 'video quality great',
 'ray video quality',
 'home video release',
 'good video quality',
 'mpi home video',
 'released home video',
 'quality video audio',
 'music video director',
 'like music video',
 'amazon instant video',
 'movie video store']

Из всех метрик я решил выбрать для биграмм jaccard, а для триграмм likelihood_ratio, так как они, судя по всему, ранжируют n-граммы с описанием именно качества продукта выше. А это информация и является наиболее полезной.

# Примеры для пяти товаров

In [77]:
five = {}
for idx, item in enumerate(entities[:5]):
    five[item] = jaccard[idx][:5] + likelihood_ratio[idx][:5]

In [78]:
df_five = pd.DataFrame(five)

In [79]:
df_five

,video,episode,trilogy,film,series
0,video game,every episode,rings trilogy,horror film,tv series
1,video quality,pilot episode,wars trilogy,great film,mini series
2,home video,final episode,prequel trilogy,first film,entire series
3,video store,first episode,matrix trilogy,see film,television series
4,video audio,last episode,lotr trilogy,film also,complete series
5,blu ray video,star trek episode,trilogy blu ray,blu ray film,series blu ray
6,video blu ray,episode star trek,star wars trilogy,film blu ray,sci fi series
7,highly recommend video,episode one best,lord rings trilogy,film special effects,series years ago
8,would recommend video,star wars episode,indiana jones trilogy,special effects film,series even though
9,video sound quality,never seen episode,dark knight trilogy,sci fi film,highly recommend series
